In [ ]:
from deep_training.data_helper import ModelArguments, TrainingArguments, DataArguments
from deep_training.nlp.models.chatglm import setup_model_profile, ChatGLMConfig
from deep_training.nlp.models.lora import LoraArguments
from transformers import HfArgumentParser

from data_utils import train_info_args, NN_DataHelper
from models import MyTransformer,ChatGLMTokenizer



In [ ]:
train_info_args['seed'] = None
parser = HfArgumentParser((ModelArguments, TrainingArguments, DataArguments, LoraArguments))
model_args, training_args, data_args, _ = parser.parse_dict(train_info_args)

setup_model_profile()

dataHelper = NN_DataHelper(model_args, training_args, data_args)
tokenizer: ChatGLMTokenizer
tokenizer, _, _, _ = dataHelper.load_tokenizer_and_config(
    tokenizer_class_name=ChatGLMTokenizer, config_class_name=ChatGLMConfig)
config = ChatGLMConfig.from_pretrained('./ckpt/lora')
config.initializer_weight = False
lora_args = LoraArguments.from_pretrained('./ckpt/lora')

assert lora_args.inference_mode == True


In [ ]:
pl_model = MyTransformer(config=config, model_args=model_args, training_args=training_args,lora_args=lora_args)
pl_model.backbone.from_pretrained(pl_model.backbone.model, pretrained_model_name_or_path = './ckpt/lora', lora_config = lora_args)
model = pl_model.get_glm_model()
model.half().cuda()
model = model.eval()

In [ ]:
ask = '请问高血压能吃党参吗'
response, history = model.chat(tokenizer, ask, history=[],max_length=512, top_p=0.7, temperature=0.99)
print(ask)
print(response)
